In [1]:
# Scripts

from models import Base
from utils import read_upload, search_books, author_overview_ddl, customer_overview_ddl
from db import get_engine, server_connection


# Libraries

from datetime import date, datetime
from typing import Optional, List
from sqlalchemy import create_engine, text, Integer, BigInteger, String, ForeignKey, Date, CheckConstraint, select, func, case, text, cast, DDL, event
from sqlalchemy.engine import URL
from sqlalchemy.orm import DeclarativeBase, Mapped, mapped_column, relationship, Session
from sqlalchemy.exc import ProgrammingError
import pandas as pd


# Setting up a server connection and creating a database

server_name = "Alstation"
database_name = "bookstore"
username = "sa"
password = "D@tabases"

server_engine = get_engine(server_name, username, password)
server_connection(server_engine, database_name)
engine = get_engine(server_name, username, password, database_name)
print(f"\nConnection to {database_name} successful!")


# Create and fill tables

views = ["author_overview", "customer_overview"]
Base.metadata.create_all(
    engine,
    tables=[table for table in Base.metadata.tables.values() if table.name not in views]
)
read_upload(Base, engine)


# Create views

with engine.connect() as conn:
    conn.execute(text("drop view if exists author_overview"))
    conn.execute(text("drop view if exists customer_overview"))
    conn.commit()
    conn.execute(author_overview_ddl)
    conn.execute(customer_overview_ddl)
    conn.commit()
    print("Views created successfully!")


Connection to bookstore successful!
Views created successfully!


In [2]:
# Access the author overview

author_overview_df = pd.read_sql(
    "select * from author_overview",
    con=engine
)
display(author_overview_df)

,id,name,age,titles,inventory
0,1,Brandon Sanderson,50 years,11 copies,158769 SEK
1,2,G.R.R. Martin,77 years,7 copies,91622 SEK
2,3,Suzanne Collins,63 years,5 copies,52523 SEK
3,4,Liu Cixin,62 years,3 copies,33722 SEK
4,5,Holly Black,54 years,3 copies,61110 SEK
5,6,Henrik Larsson,47 years,3 copies,20777 SEK
6,7,Robert Jordan,77 years,14 copies,151610 SEK
7,8,Gardner Dozois,78 years,2 copies,22155 SEK
8,9,J.R.R. Tolkien,133 years,5 copies,38306 SEK
9,10,Christopher Tolkien,101 years,1 copies,9487 SEK


In [3]:
# Access the customer overview

customer_overview_df = pd.read_sql(
    "select * from customer_overview",
    con=engine
)
display(customer_overview_df.sort_values(by="total_amount", ascending=False))

,id,customer,membership_level,total_orders,total_amount,average_rating
0,23,Ingrid Andersson,Platinum,6,3948,4.64
19,11,Sarah Jones,Platinum,5,3189,3.94
2,15,Pierre Dubois,Platinum,8,2293,4.17
12,19,Michael Williams,Gold,5,2133,3.58
14,5,John Smith,Gold,5,1713,3.50
22,8,Eva Olsson,Silver,1,947,3.50
5,6,Karin Eriksson,Silver,1,828,3.75
17,2,Anna Johansson,Silver,2,717,3.00
13,13,Lena Lindberg,Silver,2,657,3.17
11,4,Maria Nilsson,Bronze,1,538,3.00


In [4]:
# Search for books

session = Session(engine)
result = search_books(session, input("Search book: "))
display(result)

,Title,Inventory
0,The Fires of Heaven,35
1,The Path of Daggers,33
2,The Cruel Prince,42
3,The Wicked King,45
4,The Queen of Nothing,63
5,The Two Towers,44
6,The Return of the King,35
7,The Hunger Games,45
8,The Silmarillion,53
9,The Hobbit,46
